In [ ]:
import pandas as pd
import pickle
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error

from typing import List, Dict, Tuple

##### Download Data

From here: https://www.nyc.gov/site/tlc/about/tlc-trip-record-data.page

### Reading Data, Feature Engineering and Training Model

In [ ]:
def read_data(fpath):
    df = pd.read_parquet(fpath)
    return df

def preprocess_data(df: pd.DataFrame):
    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    
    return df


def feature_engineering(df):
    df['PU_DO'] = df['PULocationID'] + '_' + df['DOLocationID']
    return df


def get_features(dv: DictVectorizer, df: pd.DataFrame, categorical: List[str], numerical: List[str], type='train'):
    
    feature_dicts = df[categorical + numerical].to_dict(orient='records')
    if type == 'train':
        X = dv.fit_transform(feature_dicts)
    else:
        X = dv.transform(feature_dicts)
    return X

In [ ]:
train_data_path = "./data/green_tripdata_2021-01.parquet"
val_data_path = "./data/green_tripdata_2021-02.parquet"

In [ ]:
## Read data
train_df = read_data(train_data_path)
val_df = read_data(val_data_path)

## Preprocess data

train_df = preprocess_data(train_df)
val_df = preprocess_data(val_df)

In [ ]:
## get features

dv = DictVectorizer()
categorical = ['PU_DO'] 
numerical = ['trip_distance']

X_train = get_features(dv, train_df, categorical, numerical, type='train')
X_val = get_features(dv, val_df, categorical, numerical, type='val')

## Define target variable

target = 'duration'
y_train = train_df[target].values
y_val = val_df[target].values


In [ ]:
### Train Linear Regression Model and Save it

lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_val)

mean_squared_error(y_val, y_pred, squared=False)

with open('models/lin_reg.bin', 'wb') as f_out:
    pickle.dump((dv, lr), f_out)

In [ ]:
### Now train Lasso  Regression Models

lr = Lasso(0.01)
lr.fit(X_train, y_train)

y_pred = lr.predict(X_val)

mean_squared_error(y_val, y_pred, squared=False)

with open('models/lasso_reg.bin', 'wb') as f_out:
    pickle.dump((dv, lr), f_out)